In [1]:
    import jsonschema, asyncio, types, fnmatch, urllib, dataclasses, builtins, collections, networkx, pathlib, IPython, \
    inspect, ujson as json, io, time, mimetypes, pyld, nbformat, aiofiles, aiohttp, collections, functools, typing, sys, rdflib.extras.external_graph_libs
    from toolz.curried import *
    mime = compose(first, mimetypes.guess_type, str)
    Path = type(pathlib.Path(''))
    IPython.display.HTML("""<style>#notebook-container, .container {width: 100%;}</style>""")
    
    string_like = collections.UserString, rdflib.Namespace, rdflib.URIRef

In [2]:
    class Composition(networkx.DiGraph):
        triples = rdflib.ConjunctiveGraph()
        sessions = {}
        def weight(self, object, source, target, kwargs):
            if source == type(object): 
                if isinstance(target, Validate): return 1/len(target) if target.validate(object) else 100000
            if isinstance(target, types.FunctionType): return kwargs.get('weight', 500)
            return kwargs.get('weight', 100)
        
        async def next(self, target, object): return networkx.shortest_path(self, type(object), target, weight=curry(self.weight)(object))
        
        async def advance(self, *targets, object, composition):
            for target in targets:
                callable, target = target, target.func if isinstance(target, functools.partial) else target

                if target not in self: object = target(object); continue
                    
                try: source, next, *_ = await self.next(target, object)
                except ValueError: continue
                composition.add_edge(source, next)
                if next in {None, builtins.object}: continue
                if isinstance(next, types.ModuleType): continue
                    
                if next == target: next = callable 
                    
                object = next() if object is None else next(object)  
                if isinstance(object, types.FunctionType): object = object()                    
                if inspect.isawaitable(object): object = await object
                    
                if next == target: 
                    self.compositions.add_edge(composition, target, object=object)
                    continue
                elif isinstance(target, type):
                    if isinstance(type(object), target): 
                        self.compositions.add_edge(composition, target, object=object)
                        continue
                else:
                    try:
                        if object == target: 
                            self.compositions.add_edge(composition, target, object=object)
                            continue
                    except ValueError: ...
                        
                object = await self.advance(target, object=object, composition=composition)
            return object
        
        async def text(self, path, attr='text', **kwargs):
            if isinstance(path, pathlib.Path):
                async with aiofiles.open(path, mode='r') as file: return await file.read()
            async with self.sessions[aiohttp].get(str(path), **kwargs) as response: return await response.text()

        async def rdf(self, path, format='nt', **kwargs): 
            graph = rdflib.Graph(store=self.triples.store, identifier=path)
            graph.parse(str(path), format=format)
            return graph

        async def json(self, path, attr='text', **kwargs):
            if isinstance(path, pathlib.Path):
                async with aiofiles.open(path, mode='r') as file: return json.loads(await file.read())
            async with self.sessions[aiohttp].get(str(path), **kwargs) as response: 
                try: return await response.json(content_type=None)
                except BaseException as Exception: return Exception
        
        async def __call__(self, target, *object, keys=False, **data, ):
            global sessions
            key, value, compositions, originals = [], [], [], []
            if not object: object = None,
            if not isiterable(target): target = target,
            async with aiohttp.ClientSession() as session:
                self.sessions[aiohttp] = session
                for object in object:
                    originals.append(object)
                    object = self.prepare(object)
                    compositions.append(Composition())
                    value.append(self.advance(*target, object=object, composition=compositions[-1]))
                    try: self.compositions.add_edge(object, compositions[-1])
                    except TypeError: ...
                value = await asyncio.gather(*value)
            try: 
                if keys: return dict(zip(originals, value))
            except TypeError: ...
            return value
        
        
        def __add__(self, object, *types): 
            self.add_path([Forward[object] if isinstance(object, str) else object for object in object])
            return self

        @staticmethod
        def prepare(object):
            if isinstance(object, str):
                original = str
                if urllib.parse.urlparse(object).scheme in {'http', 'https'}: object = rdflib.Namespace(object)
                else:
                    try: 
                        path = pathlib.Path(object)
                        if path.exists(): object = path
                        else: object = collections.UserString(object)
                    except:
                        forward = Forward[object]
                        if forward: object = forward
            return object

    Composition.compositions = Composition()

In [3]:
    def yaml(object):
        try: import yaml
        except: from ruamel import yaml
        return yaml.safe_load(__import__('io').StringIO(str(object)))
            

In [4]:
    def new(cls, *args, _root=False): return type.__new__(cls, *args)
    typing.TypingMeta.__new__ = new
    def __getitem__(cls, object): return cls(object)
    typing.TypingMeta.__getitem__ = __getitem__


    class ForwardRef(getattr(typing, 'ForwardRef', getattr(typing, '_ForwardRef'))):
        def __bool__(self): 
            try: self()
            except: ...
            return self.__forward_evaluated__
        
        def __call__(self, *object, globals=sys.modules, locals=sys.modules):
            if self.__forward_evaluated__: 
                if callable(self.__forward_value__):
                    if object: return self.__forward_value__(*object)
                    else: return self.__forward_value__
                return self.__forward_value__
            try:
                self.__forward_value__, self.__forward_evaluated__ = eval(self.__forward_arg__, globals, locals), True
                return self(*object, globals=globals, locals=locals)
            except: ...

        def __hash__(self): 
            self()
            try: return hash(self.__forward_value__ if self.__forward_evaluated__ else self.__forward_arg__)
            except: return hash(self.__forward_arg__)
                
        def __eq__(self, object):
            self();
            try:
                if hash(self) == hash(object): return True
            except: 
                if self.__forward_arg__ == object: return True
            return False
        
    Forward = ForwardRef

In [5]:
    @dataclasses.dataclass
    class Validate:
        data: object = ''
        callable: callable = None
        def __hash__(self): return hash(str(self.data))
        def __len__(self): return len(self.data)
        async def __call__(self, object): 
            if self.callable:
                if isinstance(object, string_like): object = str(object)
                object = self.callable(object) 
                if inspect.isawaitable(object): object = await object
            return object
        
    class Mime(Validate, collections.UserString):
        def validate(self, object): 
            if isinstance(object, type(self)): return hash(object) == hash(self)
            return not self.data or self.data == mime(str(object))
        

In [6]:
    class Pattern(collections.UserList, Validate): 
        def __init__(self, *args, **kwargs):
            self.callable = kwargs.pop('callable', None)
            super().__init__(*args, **kwargs)
        def validate(self, object): return any(fnmatch.fnmatch(str(object), pattern) for pattern in self.data)
        def __hash__(self): return hash(tuple(self.data))
        

In [7]:
    class Schema(jsonschema.Draft4Validator, Validate):
        def __init__(self, *args, **kwargs):
            self.callable = kwargs.pop('callable', None)
            super().__init__(*args, **kwargs)
            
        @property
        def data(self): return self.schema
        def validate(self, object):
            try: return super().validate(object) or True
            except jsonschema.ValidationError: return False
        
        @classmethod
        def new(cls, type, **kwargs):
            class Model(__import__('pydantic').BaseModel): object: type
            schema = json.loads(Model.schema_json())['properties']['object']
            return cls(schema.pop('title') and schema, **kwargs)
        
        def __len__(self): return len(pyld.jsonld.compact({'@context': {'@vocab': '://',
                                     'definitions': '@nest', 'properties': '@nest'
                                     }, **nbformat.validator._get_schema_json(nbformat.v4)}, {'@vocab': '://'}).keys())

In [8]:
    def series_to_frame(series):
        if series.dtypes == __import__('pandas').np.dtype('O'): return series.apply(__import__('pandas').Series)
        return series.to_frame()

In [9]:
    mimetypes.add_type('text/markdown', '.md')
    mimetypes.add_type('text/markdown', '.markdown')
    mimetypes.add_type('application/x-sqlite3', '.sqlite')
    mimetypes.add_type('application/x-yaml', '.yml')
    mimetypes.add_type('application/x-yaml', '.yaml')
    mimetypes.add_type('application/x-ipynb+json', '.ipynb')

In [10]:
    @curry
    def convert(type, nb): 
        for cell in nb.cells: cell.source = ''.join(cell.source)
        return __import__('nbconvert').get_exporter(type)().from_notebook_node(nb)[0]

In [14]:
    ado = Composition()
    JSON = Mime('application/json', callable=ado.json)
    ado + [Composition, networkx.to_directed, networkx.DiGraph]
    ado + [rdflib.Namespace, compose(partial(
        IPython.display.IFrame, width='100%', height='600px'
    ),str), IPython.display.IFrame]
    
    ado + [rdflib.Namespace, ado.json, dict] + [ado.json, list] + [ado.json, float] + [ado.json, int]
    ado + [rdflib.Namespace, ado.text, str]
    
    ado + [Path, Mime('text/html', callable=compose(IPython.display.HTML, str)), IPython.display.HTML]
    ado + [Path, Mime('text/markdown', callable=compose(IPython.display.Markdown, str)), IPython.display.Markdown]
    ado + [Path, Mime('application/json', callable=compose(IPython.display.JSON, str)), IPython.display.JSON]   
    
    ado + [Path, Mime('application/x-yaml', callable=ado.text), str, yaml] 
    ado + [collections.UserString, yaml] + [yaml, dict, IPython.display.JSON] + [yaml, list, IPython.display.JSON] + [yaml, float] + [yaml, int] 
    ado + [yaml, type(None)] + [yaml, str]
    ado + [Path, JSON, dict] + [JSON, list] + [JSON, float] + [JSON, int]
    ado + [Mime('application/json', callable=ado.text), str]
        
    
    ado + [Path, Mime('text/csv', callable=Forward['pandas.read_csv']), 'pandas.DataFrame']
    ado + [rdflib.Namespace, Mime('text/csv', callable=Forward['pandas.read_csv']), 'pandas.DataFrame']
    ado + [list, Schema.new(typing.List[typing.Union[int, None, float, str]], callable=Forward['pandas.Series']), 'pandas.Series']
    ado + ['pandas.Series', series_to_frame, 'pandas.DataFrame']
    
    ado + [list, Schema.new(typing.List[typing.Union[list, dict]], callable=Forward['pandas.DataFrame']), 'pandas.DataFrame', 'pandas.DataFrame.transpose']
    ado + [dict, 'pandas.Series']
    
    ado + [Path, Mime('application/x-ipynb+json', callable=ado.json), dict, 'nbformat.from_dict', 'nbformat.NotebookNode']
    ado + ['nbformat.NotebookNode', convert('html'), str, IPython.display.HTML]
    
    ado + [Path, Mime('image/png', callable=compose(IPython.display.Image, str)), IPython.display.Image]
    ado + [Path, Mime('image/jpeg', callable=compose(IPython.display.Image, str)), IPython.display.Image]  
    ado + [Path, Mime('image/bmp', callable=compose(IPython.display.Image, str)), IPython.display.Image]
    ado + [Path, Mime('image/svg+xml', callable=compose(IPython.display.SVG, str)), IPython.display.SVG]   
    
    ado + [Path, Mime('image/png', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray']
    ado + [Path, Mime('image/jpeg', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray']
    ado + [Path, Mime('image/bmp', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray', list]

    ado + [rdflib.Namespace, Mime('image/png', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray']
    ado + [rdflib.Namespace, Mime('image/jpeg', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray']
    ado + [rdflib.Namespace, Mime('image/bmp', callable=Forward('skimage.io.imread')), 'pandas.np.ndarray', list]

    
    ado + [dict, Schema.new(typing.Dict[str, list], callable=partial(networkx.from_dict_of_lists, create_using=networkx.DiGraph)), networkx.DiGraph]
    ado + [dict, Schema.new(typing.Dict[str, dict], callable=partial(networkx.from_dict_of_dicts, create_using=networkx.DiGraph)), networkx.DiGraph]
    ado + [dict, Schema.new(typing.Dict[str, list], callable=networkx.from_dict_of_lists), networkx.Graph]
    ado + [dict, Schema.new(typing.Dict[str, dict], callable=networkx.from_dict_of_dicts), networkx.Graph]
    
    [ado + [object, IPython.display.display] for object in list(ado.node) if isinstance(object, type) and issubclass(object, IPython.display.DisplayObject)]
    ado + [networkx.DiGraph, networkx.nx_pydot.to_pydot] + [networkx.Graph, networkx.nx_pydot.to_pydot, 'pydot.Dot', 'graphviz.Source']

    ado + [Forward, operator.attrgetter('__forward_value__'), dict]
    ado + [Forward, operator.attrgetter('__forward_value__'), list]
    ado + [type(None), 'pandas.util.testing.makeDataFrame', 'pandas.DataFrame']
    
    ado + [rdflib.Namespace, ado.rdf, rdflib.Graph, rdflib.extras.external_graph_libs.rdflib_to_networkx_digraph, networkx.DiGraph]
    ado + [rdflib.Graph, rdflib.extras.external_graph_libs.rdflib_to_networkx_graph, networkx.Graph]
    